# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The Nvidia Blackwell GPU features advanced Transformer Engine, confidential computing, and high-speed NVLink. It targets generative AI and large-scale AI models. It uses the 4NP process technology.'

## Regular search

In [16]:
# choose location (try to change to your own city!)

city = "Amsterdam"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [17]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

/var/folders/r2/n1vtxc157gz88z42zprlblb00000gn/T/ipykernel_26132/2852339428.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


https://www.accuweather.com/en/nl/amsterdam/249758/current-weather/249758
https://www.weather25.com/europe/netherlands/noord-holland/amsterdam?page=today
https://www.yr.no/en/forecast/hourly-table/2-2759794/The+Netherlands/North+Holland/Gemeente+Amsterdam/Amsterdam?i=2
https://meteostate.com/forecast/nl/amsterdam/
https://www.bbc.com/weather/2759794
https://www.accuweather.com/en/nl/amsterdam/249758/weather-forecast/249758


In [18]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [19]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://www.accuweather.com/en/nl/amsterdam/249758/current-weather/249758


<body class="current-weather ads-not-loaded full-animation rfphrase-disabled">
<div class="template-root" style="display:none">
<div class="basic-header" style="visibility: hidden;">
<a class="pwa-top-banner" href="/pwa">
<svg class="arrow-icon" height="18" viewbox="0 0 18 18" width="18" xmlns="http://www.w3.org/2000/svg"><defs><path d="m8.495.505 5 5v.99l-5 5-.99-.99 3.805-3.806L0 6.7V5.3l11.31-.001-3.805-3.804.99-.99z" id="a"></path></defs><use fill="#fff" fill-rule="nonzero" transform="translate(2 3)" xlink:href="#a"></use></svg>
		Go Back
	</a>
<div class="nav-alert-container no-banners">
<div class="afb-nav-container" id="afb-nav-container">
<div class="afb-nav-items">
<ul>
<li>
<a class="afb-white" href="https://afb.accuweather.com" noreferrer&quot;="" rel='"noopener' target='"_blank"'>
							For Business
						</a>
</li>
<li>|</li>
<li>
<a href="https://afb.accuweather.com/accuweather-skyguard"

In [20]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://www.accuweather.com/en/nl/amsterdam/249758/current-weather/249758


Amsterdam, North Holland Amsterdam North Holland Around the Globe Around the Globe Hurricane Tracker Severe Weather Radar & Maps News & Features Astronomy Business Climate Health Recreation Sports Travel Warnings Data Suite Forensics Advertising Superior Accuracy™ Video Current Weather 10:04 PM Night Wind E 9 km/h Wind Gusts 19 km/h Probability of Precipitation 1% Probability of Thunderstorms 0% Precipitation 0.0 mm Cloud Cover 25% Evening Overnight Sun & Moon Temperature History Further Ahead Further Ahead Hourly Daily Monthly Around the Globe Around the Globe Hurricane Tracker Severe Weather Radar & Maps News Video Top Stories Hurricane First Atlantic hurricane of the 2025 season may form around mid-August 2 hours ago Weather Forecasts Fall forecast 2025: Warmth to fuel fires, storms before chill hits US 4 days ago Weather News Canyon Fire joins California’s escalating wildfire crisis 2 days ago Astr

## Agentic Search

In [21]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Amsterdam', 'region': 'North Holland', 'country': 'Netherlands', 'lat': 52.374, 'lon': 4.8897, 'tz_id': 'Europe/Amsterdam', 'localtime_epoch': 1754856312, 'localtime': '2025-08-10 22:05'}, 'current': {'last_updated_epoch': 1754856000, 'last_updated': '2025-08-10 22:00', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 55, 'wind_dir': 'NE', 'pressure_mb': 1027.0, 'pressure_in': 30.33, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 0, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 15.4, 'windchill_f': 59.7, 'heatindex_c': 15.4, 'heatindex_f': 59.7, 'dewpoint_c': 10.5, 'dewpoint_f': 50.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 12.5, 'gust_kph': 20.2}}


In [22]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "Amsterdam",
        "region": "North Holland",
        "country": "Netherlands",
        "lat": 52.374,
        "lon": 4.8897,
        "tz_id": "Europe/Amsterdam",
        "localtime_epoch": 1754856312,
        "localtime": "2025-08-10 22:05"
    },
    "current": {
        "last_updated_epoch": 1754856000,
        "last_updated": "2025-08-10 22:00",
        "temp_c": 17.2,
        "temp_f": 63.0,
        "is_day": 0,
        "condition": {
            "text": "Clear",
            "icon": "//cdn.weatherapi.com/weather/64x64/night/113.png",
            "code": 1000
        },
        "wind_mph": 6.0,
        "wind_kph": 9.7,
        "wind_degree": 55,
        "wind_dir": "NE",
        "pressure_mb": 1027.0,
        "pressure_in": 30.33,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 72,
        "cloud": 0,
        "feelslike_c": 17.2,
        "feelslike_f": 63.0,
        "windchill_c": 15.4,
        "windchill_f": 59.7,
     

<img src="./google_sample.png" width="800" height="600">